In [313]:
import pandas as pd
import statistics
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

In [314]:
df = pd.read_csv("small3insert.csv")
df["Timestamp"] = pd.to_datetime(df["Timestamp"])

In [315]:
df.head()

,Case,Event,Activity,Timestamp,Resource,Resource_failure_rate,Resource_Pass/Fail,order,resource_anomaly_type,resource_parameter,trace_change_resource,variant_num
0,case_0,9367,Activity A,1970-01-01 09:00:00,Resource_Group2_res_1,0.026984,0,1,normal,NaN,0,var_358
1,case_0,9368,Activity B,1970-01-01 10:00:00,Resource_Group0_res_0,0.018454,0,2,normal,NaN,0,var_358
2,case_0,9369,Activity C,1970-01-01 11:00:00,Resource_Group1_res_0,0.002551,0,3,normal,NaN,0,var_358
3,case_0,9370,Activity D,1970-01-01 12:00:00,Resource_Group2_res_2,0.029775,0,4,normal,NaN,0,var_358
4,case_0,9371,Activity E,1970-01-01 13:00:00,Resource_Group0_res_0,0.018454,0,5,normal,NaN,0,var_358


In [316]:
df = df[["Case","Activity","Timestamp","Resource","resource_anomaly_type","Event"]]

In [317]:
df.columns = ["caseid","activity", "ts","resource","outcome","event"]

In [318]:
list_ = []
for i in range(len(df)-1):
    if df.iloc[i+1]["caseid"] == df.iloc[i]["caseid"]:
        list_ = list_ + [df.iloc[i+1]["ts"]-df.iloc[i]["ts"] ]
    else:
        list_ = list_ + [list_[-1]]
list_ = list_ + [list_[-1]]
df["t"] = list_

In [319]:
df.head()

,caseid,activity,ts,resource,outcome,event,t
0,case_0,Activity A,1970-01-01 09:00:00,Resource_Group2_res_1,normal,9367,0 days 01:00:00
1,case_0,Activity B,1970-01-01 10:00:00,Resource_Group0_res_0,normal,9368,0 days 01:00:00
2,case_0,Activity C,1970-01-01 11:00:00,Resource_Group1_res_0,normal,9369,0 days 01:00:00
3,case_0,Activity D,1970-01-01 12:00:00,Resource_Group2_res_2,normal,9370,0 days 01:00:00
4,case_0,Activity E,1970-01-01 13:00:00,Resource_Group0_res_0,normal,9371,0 days 01:00:00


In [320]:
def first_L_events(df,L):
    df_t = df.groupby(["caseid"]).agg(list).reset_index()[["caseid","event"]]

    list_len = df_t["event"].map(lambda x: len(x))
    df_t["len"] = list_len
    df_L = df_t.loc[df_t['len'] >= L]

    list_case = df_L["event"].map(lambda x: x[:L])

    out = pd.DataFrame(columns = ["caseid","list_case"])    

    
    output = df_L["caseid"]
    output["list_case"] = [i for i in list_case]

    
    return output
 
    
    

In [321]:
first_L_events(df,3)

0                                                       case_0
1                                                       case_1
2                                                      case_10
3                                                     case_100
4                                                    case_1000
                                   ...                        
4996                                                  case_996
4997                                                  case_997
4998                                                  case_998
4999                                                  case_999
list_case    [[9367, 9368, 9369], [13225, 13226, 13227], [1...
Name: caseid, Length: 5001, dtype: object

In [322]:
def first_L_events(df,L):
    df_t = df.groupby(["caseid"]).agg(list).reset_index()[["caseid","event"]]

    list_len = df_t["event"].map(lambda x: len(x))
    df_t["len"] = list_len
    df_L = df_t.loc[df_t['len'] >= L] 
    df_L["event_prefix"] = df_L["event"].map(lambda x: x[0:L])
    return df_L[["caseid","event_prefix"]]
 
    
    

In [323]:
first_L_events(df, 5)

,caseid,event_prefix
0,case_0,"[9367, 9368, 9369, 9370, 9371]"
1,case_1,"[13225, 13226, 13227, 13227, 13227]"
2,case_10,"[1025, 1026, 1027, 1028, 1029]"
3,case_100,"[39553, 39554, 39555, 39556, 39557]"
4,case_1000,"[27951, 27952, 27953, 27954, 27955]"
...,...,...
4995,case_995,"[11080, 11081, 11082, 11083, 11084]"
4996,case_996,"[9536, 9537, 9538, 9539, 9540]"
4997,case_997,"[7567, 7568, 7569, 7570, 7571]"
4998,case_998,"[8949, 8950, 8951, 8952, 8953]"


In [349]:
def aggregation_encoding(df):
    def avg(x):
        avg = x[0]
        for i in x[1:]:
            avg = avg + i
        return avg/len(x)
    
    
    df_grouped = df.groupby(["caseid"]).agg(list)
    for activity in df["activity"].unique():
        df_grouped[activity] = df_grouped["activity"].map(lambda x : x.count(activity)/len(x))

    for activity in df["resource"].unique():
        df_grouped[activity] = df_grouped["resource"].map(lambda x : x.count(activity)/len(x))
    df_grouped["avg_t"] = df_grouped["t"].map(lambda x : avg(x))


    return df_grouped[df_grouped.columns[6:]].copy()
    

In [350]:
a = aggregation_encoding(df)

45847

In [167]:
df_grouped= df.groupby(["caseid"]).agg(list)

In [168]:
max_lenght = df_grouped["activity"].map(lambda x: len(x)).max()

In [169]:
df_grouped["max_lenght"] = max_lenght

In [266]:
def index_encoding(df):
    df_grouped= df.groupby(["caseid"]).agg(list)
    max_lenght = df_grouped["activity"].map(lambda x: len(x)).max()
    
    for i in range(0,max_lenght):
        for act in df["activity"].unique():
            df_grouped[f"{act}_{i+1}"] = df_grouped["activity"].map(lambda x : 1 if act in x else 0).copy()
        for resource in df["resource"].unique():
            df_grouped[f"{resource}_{i+1}"] = df_grouped["resource"].map(lambda x : 1 if resource in x else 0).copy() 
    
        t_i = []
    
        for index in range(0,len(df_grouped)):
            try:
                t_i.append(df_grouped.iloc[index]["ts"][i])
            except:
                t_i.append(0)
        
        df_grouped[f"t_{i+1}"] = t_i

    df_grouped["out"] = df_grouped["outcome"].map(lambda x : x[0])
    return df_grouped[df_grouped.columns[6:]].copy()

In [273]:
a  =index_encoding(df)

### Random Forest

In [352]:
df_rf = aggregation_encoding(df)

In [354]:
df_rf["avg_t"] = df_rf["avg_t"].map(lambda x : x.total_seconds())

In [355]:
df_grouped= df.groupby(["caseid"]).agg(list)
df_rf["out"] = df_grouped["outcome"].map(lambda x : x[0])

In [356]:
events = first_L_events(df,2)

In [357]:
XY = df_rf[df_rf.index.isin(events["caseid"])]

In [358]:
labels = XY.pop('out').copy()
data = XY.copy()

In [359]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3)

In [360]:
clf = RandomForestClassifier(max_depth=5, oob_score=True)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, oob_score=True)

In [361]:
prediction = clf.predict(X_test)

In [367]:
from sklearn.metrics import accuracy_score

In [368]:
accuracy_score(y_test, prediction)

1.0

### Decision Tree

In [369]:
clf = RandomForestClassifier(max_depth=5, oob_score=True,n_estimators=1)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, n_estimators=1, oob_score=True)

In [370]:
prediction = clf.predict(X_test)

In [371]:
accuracy_score(y_test, prediction)

1.0